# Game servers autopilot
Multiplayer game publishers often need to either over-provision resources or manually manage compute resource allocation when launching a large-scale worldwide game, to avoid the long player-wait in the game lobby. Game publishers need to develop, config, and deploy tools that helped them to monitor and control the compute allocation.

This notebook demonstrates Game server autopilot, a new machine learning-based example tool that makes it easy for game publishers to reduce the time players wait for compute to spawn, while still avoiding compute over-provisioning. It also eliminates manual configuration decisions and changes publishers need to make and reduces the opportunity for human errors.

We heard from customers that optimizing compute resource allocation is not trivial. This is because it often takes substantial time to allocate and prepare EC2 instances. The time needed to spin up an EC2 instance and install game binaries and other assets must be learned and accounted for in the allocation algorithm. Ever-changing usage patterns require a model that is adaptive to emerging player habits. Finally, the system also performs scale down in concert with new server allocation as needed.

We describe a reinforcement learning-based system that learns to allocate resources in response to player usage patterns. The hosted model directly predicts the required number of game-servers so as to allow EKS the time to allocate instances to reduce player wait time. The training process integrates with the game eco-system, and requires minimal manual configuration.

## Pre-requisites 

### Imports

To get started, we'll import the Python libraries we need, set up the environment with a few prerequisites for permissions and configurations.

In [ ]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
import numpy as np
from IPython.display import HTML
import time
from time import gmtime, strftime

sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework

### Setup S3 bucket

Set up the linkage and authentication to the S3 bucket that you want to use for checkpoint and the metadata. 

In [ ]:
sage_session = sagemaker.session.Session()
s3_bucket = sage_session.default_bucket()
s3_output_path = "s3://{}/".format(s3_bucket)
print("S3 bucket path: {}".format(s3_output_path))

### Parameters

Adding new parameters for the job require update in the training section that invokes the RLEstimator.

In [ ]:
job_name_prefix = "rl-game-server-autopilot"
job_duration_in_seconds = 60 * 60 * 24 * 5
train_instance_count = 1
cloudwatch_namespace = "rl-game-server-autopilot"
min_servers = 10
max_servers = 100
# over provisionning factor. use 5 for optimal.
over_prov_factor = 5
# gamma is the discount factor
gamma = 0.9
# if local inference is set gs_inventory_url=local and populate learning_freq
gs_inventory_url = "https://4bfiebw6ui.execute-api.us-west-2.amazonaws.com/api/currsine1h/"
# gs_inventory_url = 'local'
# sleep time in seconds between step() executions
learning_freq = 65
# actions are normelized between 0 and 1, action factor the number of game servers needed e.g. 100 will be 100*action and clipped to the min and max servers parameters above
action_factor = 100

In [ ]:
# Pick the instance type
instance_type = "ml.c5.xlarge"  # 4 cpus
#     instance_type = "ml.c5.4xlarge" #16 cpus
#      instance_type = "ml.c5.2xlarge" #8 cpus
#      instance_type = "ml.c4.4xlarge"
#     instance_type = "ml.p2.8xlarge" #32 cpus
#     instance_type = "ml.p3.2xlarge" #8 cpus
#    instance_type = "ml.p3.8xlarge" #32 cpus
#     instance_type = "ml.p3.16xlarge" #96 cpus
#     instance_type = "ml.c5.18xlarge" #72 cpus

num_cpus_per_instance = 4

### Create an IAM role

Either get the execution role when running from a SageMaker notebook instance `role = sagemaker.get_execution_role()` or, when running from local notebook instance, use utils method `role = get_execution_role()` to create an execution role. In this example, the env thru the training job, publishes cloudwatch custom metrics as well as put values in DynamoDB table. Therefore, an appropriate role is required to be set to the role arn below.

In [ ]:
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

# Set up the environment

The environment is defined in a Python file called gameserver_env.py and the file is uploaded on /src directory.
The environment also implements the init(), step() and reset() functions that describe how the environment behaves. This is consistent with Open AI Gym interfaces for defining an environment. It also implements help functions for custom CloudWatch metrics (populate_cloudwatch_metric()) and a simple sine demand simulator (get_curr_sine1h())

1. init() - initialize the environment in a pre-defined state
2. step() - take an action on the environment
3. reset()- restart the environment on a new episode
4. get_curr_sine1h() - return the sine value based on the current second.
5. populate_cloudwatch_metric(namespace,metric_value,metric_name) - populate the metric_name with metric_value in namespace. 



In [ ]:
!pygmentize src/gameserver_env.py

## Configure the presets for RL algorithm

The presets that configure the RL training jobs are defined in the train_gameserver_ppo.py file which is also uploaded on the /src directory. Using the preset file, you can define agent parameters to select the specific agent algorithm. You can also set the environment parameters, define the schedule and visualization parameters, and define the graph manager. The schedule presets will define the number of heat up steps, periodic evaluation steps, training steps between evaluations.
It can be used to define custom hyperparameters.

In [ ]:
!pygmentize src/train_gameserver_ppo.py

## Train the RL model using the Python SDK Script mode

The RLEstimator is used for training RL jobs. 

1. The entry_point value indicates the script that invokes the GameServer RL environment.
2. source_dir indicates the location of environment code which currently includes train-gameserver-ppo.py and game_server_env.py. 
3. Specify the choice of RL toolkit and framework. This automatically resolves to the ECR path for the RL Container. 
4. Define the training parameters such as the instance count, job name, S3 path for output and job name. 
5. Specify the hyperparameters for the RL agent algorithm. The RLCOACH_PRESET or the RLRAY_PRESET can be used to specify the RL agent algorithm you want to use. 
6. Define the metrics definitions that you are interested in capturing in your logs. These can also be visualized in CloudWatch and SageMaker Notebooks. 

In [ ]:
metric_definitions = [
    {
        "Name": "episode_reward_mean",
        "Regex": "episode_reward_mean: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)",
    },
    {
        "Name": "episode_reward_max",
        "Regex": "episode_reward_max: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)",
    },
    {
        "Name": "episode_len_mean",
        "Regex": "episode_len_mean: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)",
    },
    {"Name": "entropy", "Regex": "entropy: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)"},
    {
        "Name": "episode_reward_min",
        "Regex": "episode_reward_min: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)",
    },
    {"Name": "vf_loss", "Regex": "vf_loss: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)"},
    {"Name": "policy_loss", "Regex": "policy_loss: ([-+]?[0-9]*\\.?[0-9]+([eE][-+]?[0-9]+)?)"},
]

metric_definitions

In [ ]:
%%time
# metric_definitions = RLEstimator.default_metric_definitions(RLToolkit.RAY)

estimator = RLEstimator(
    entry_point="train_gameserver_ppo.py",
    source_dir="src",
    dependencies=["common/sagemaker_rl"],
    toolkit=RLToolkit.RAY,
    toolkit_version="0.6.5",
    framework=RLFramework.TENSORFLOW,
    role=role,
    instance_type=instance_type,
    instance_count=train_instance_count,
    output_path=s3_output_path,
    base_job_name=job_name_prefix,
    metric_definitions=metric_definitions,
    max_run=job_duration_in_seconds,
    hyperparameters={
        "cloudwatch_namespace": cloudwatch_namespace,
        "gs_inventory_url": gs_inventory_url,
        "learning_freq": learning_freq,
        "time_total_s": job_duration_in_seconds,
        "min_servers": min_servers,
        "max_servers": max_servers,
        "gamma": gamma,
        "action_factor": action_factor,
        "over_prov_factor": over_prov_factor,
        "save_model": 1,
    },
)

estimator.fit(wait=False)
job_name = estimator.latest_training_job.job_name
print("Training job: %s" % job_name)

## Store intermediate training output and model checkpoints

The output from the training job above is stored in a S3.

In [ ]:
%%time

job_name = estimator._current_job_name
print("Job name: {}".format(job_name))

s3_url = "s3://{}/{}".format(s3_bucket, job_name)

output_tar_key = "{}/output/output.tar.gz".format(job_name)

intermediate_folder_key = "{}/output/intermediate/".format(job_name)
output_url = "s3://{}/{}".format(s3_bucket, output_tar_key)
intermediate_url = "s3://{}/{}".format(s3_bucket, intermediate_folder_key)

print("S3 job path: {}".format(s3_url))
print("Output.tar.gz location: {}".format(output_url))
print("Intermediate folder path: {}".format(intermediate_url))

tmp_dir = "/tmp/{}".format(job_name)
os.system("mkdir {}".format(tmp_dir))
print("Create local folder {}".format(tmp_dir))

## Evaluation of RL models
We use the latest checkpointed model to run evaluation for the RL Agent.

### Load checkpointed model

Checkpointed data from the previously trained models will be passed on for evaluation / inference in the checkpoint channel. 
Since TensorFlow stores ckeckpoint file containes absolute paths from when they were generated (see issue), we need to replace the absolute paths to relative paths. This is implemented within evaluate-game-server.py

In [ ]:
%%time

wait_for_s3_object(s3_bucket, output_tar_key, tmp_dir)

if not os.path.isfile("{}/output.tar.gz".format(tmp_dir)):
    raise FileNotFoundError("File output.tar.gz not found")
os.system("tar -xvzf {}/output.tar.gz -C {}".format(tmp_dir, tmp_dir))

checkpoint_dir = "{}/checkpoint".format(tmp_dir)

print("Checkpoint directory {}".format(checkpoint_dir))

In [ ]:
%%time
checkpoint_path = "s3://{}/{}/checkpoint/".format(s3_bucket, job_name)
if not os.listdir(checkpoint_dir):
    raise FileNotFoundError("Checkpoint files not found under the path")
os.system("aws s3 cp --recursive {} {}".format(checkpoint_dir, checkpoint_path))
print("S3 checkpoint file path: {}".format(checkpoint_path))

## Run the evaluation step
Use the checkpointed model to run the evaluation step.

In [ ]:
%%time
job_name = "5obs-local-sine-2019-08-18-21-13-45-314"
print("job_name: %s" % job_name)
estimator_eval = RLEstimator(
    entry_point="evaluate_gameserver_ppo.py",
    source_dir="src",
    dependencies=["common/sagemaker_rl"],
    role=role,
    toolkit=RLToolkit.RAY,
    toolkit_version="0.6.5",
    framework=RLFramework.TENSORFLOW,
    instance_type=instance_type,
    instance_count=1,
    base_job_name=job_name_prefix + "-evaluation",
    hyperparameters={
        "cloudwatch_namespace": cloudwatch_namespace,
        "gs_inventory_url": gs_inventory_url,
        "learning_freq": learning_freq,
        "time_total_s": job_duration_in_seconds,
        "min_servers": min_servers,
        "max_servers": max_servers,
        "gamma": gamma,
        "action_factor": action_factor,
        "over_prov_factor": over_prov_factor,
        "save_model": 1,
    },
)
estimator_eval.fit({"model": checkpoint_path})
job_name = estimator_eval.latest_training_job.job_name
print("Evaluation job: %s" % job_name)

## Hosting
Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same insantance (or type of instance) that we used to train. The endpoint deployment can be accomplished as follows:

### Model deployment

Now let us deploy the RL policy so that we can get the optimal action, given an environment observation.
In case the notebook restarted and lost its previous estimator object, populate the estimator.model_data with the full s3 link to the model.tar.gz. e.g., s3://sagemaker-us-west-2-356566070122/rl-gameserver-autopilot-2019-07-19-19-36-32-926/output/model.tar.gz

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel

print("model name: %s" % estimator.model_data)
model_data = "s3://sagemaker-us-west-2-356566070122/rl-gs-training-2019-09-23-15-41-40-260/output/model.tar.gz"
model = TensorFlowModel(model_data=model_data, framework_version="2.1.0", role=role)

predictor = model.deploy(initial_instance_count=1, instance_type=instance_type)

### Inference
Now that the trained model is deployed at an endpoint that is up-and-running, we can use this endpoint for inference. The format of input should match that of observation_space in the defined environment. In this example, the observation space is a 15 dimensional vector formulated from previous and current observations. For the sake of space, this demo doesn't include the non-trivial construction process. Instead, we provide a dummy input below. For more details, please check src/gameserver_env.py.

In [ ]:
sagemaker_region = "us-west-2"
sagemaker_client = boto3.client("sagemaker-runtime", region_name=sagemaker_region)
# populate the correct endpoint_name
endpoint_name = "sagemaker-tensorflow-serving-2019-09-23-20-53-20-237"
content_type = "application/json"
accept = "Accept"
last_observations = np.arange(1, 16)

response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType=content_type, Accept=accept, Body=last_observations
)
response["Body"].read()

### Delete the Endpoint

Having an endpoint running will incur some costs. Therefore as a clean-up job, we should delete the endpoint.

In [ ]:
predictor.delete_endpoint()